In [2]:
from llama_cpp import Llama, Completion, LlamaTokenizer

In [ ]:
# let's translate the text of the first chapter
with open("../data/books/three_body/chapters/1.json", "r") as f:
    chapter_1_json = json.loads(f.read())


In [8]:
llm = Llama(
    model_path="/home/brian/github/llama.cpp/models/7B/Chinese-Alpaca-2/ggml-model-q4_0.bin",
    n_ctx=4096,
    n_gpu_layers=30
)

llama.cpp: loading model from /home/brian/github/llama.cpp/models/7B/Chinese-Alpaca-2/ggml-model-q4_0.bin
llama_model_load_internal: format     = ggjt v3 (latest)
llama_model_load_internal: n_vocab    = 55296
llama_model_load_internal: n_ctx      = 4096
llama_model_load_internal: n_embd     = 4096
llama_model_load_internal: n_mult     = 5504
llama_model_load_internal: n_head     = 32
llama_model_load_internal: n_head_kv  = 32
llama_model_load_internal: n_layer    = 32
llama_model_load_internal: n_rot      = 128
llama_model_load_internal: n_gqa      = 1
llama_model_load_internal: rnorm_eps  = 1.0e-06
llama_model_load_internal: n_ff       = 11008
llama_model_load_internal: freq_base  = 10000.0
llama_model_load_internal: freq_scale = 1
llama_model_load_internal: ftype      = 2 (mostly Q4_0)
llama_model_load_internal: model size = 7B
llama_model_load_internal: ggml ctx size =    0.08 MB
llama_model_load_internal: using CUDA for GPU acceleration
llama_model_load_internal: mem required  = 10

In [14]:
def translate_paragraph(text, llm, use_example=None, method=None):
    model_path = "/home/brian/github/llama.cpp/models/7B/Chinese-Alpaca-2/ggml-model-q4_0.bin"
    # system_prompt="You will be provided with Chinese language text. Please respond with the English translation of the text. 你是翻译者,请把输入的段子翻译成英文。"
    system_prompt="Translate from Chinese to English, only reply in English 把中文翻译成英文，只用英文回答问题"
    n_ctx = 4096
    llm = Llama(model_path=model_path, n_ctx=n_ctx)

    # message components
    messages = []

    # system prompt
    system_prompt = {"role": "system", "content": system_prompt}
    messages.append(system_prompt)

    # examples to help the completion get a better sense of how to translate
    if use_example:
        messages.append({"role": "user", "content": "在那个已被忘却的日子里，它的世界颠覆了。泥土飞走，出现了一条又深又宽的峡谷，然后泥土又轰隆隆地飞回来，峡谷消失了，在原来峡谷的尽头出现了一座黑色的孤峰。其实，在这片广阔的疆域上，这种事常常发生，泥土飞走又飞回，峡谷出现又消失，然后是孤峰降临，好像是给每次灾变打上一个醒目的标记。褐蚁和几百个同族带着幸存的蚁后向太阳落下的方向走了一段路，建立了新的帝国。"})
        messages.append({"role": "assistant", "content": "In that forgotten day, its world was turned upside down. The soil flew away and a deep and wide canyon appeared, then the soil flew back and the canyon disappeared, at the end of which appeared a black peak. In fact, such things often happened in this vast domain, where the soil flew away and flew back, then there was a canyon, then a peak, as if to mark each disaster. The brown ants and hundreds of their kind with surviving queens walked towards the direction where the sun set and established a new empire."})

    messages.append({"role": "user", "content": text})
    if method == "completion":
        completion_prompt = f"\n\n### Chinese:\n${text}\n\n### English:\n"
        stop = ["\n","###"]
        response = llm.create_completion(max_tokens=5000,model=model_path, prompt=completion_prompt, stop=stop)
        print(response["choices"][0]["text"])
        return response
    else:
        response = llm.create_chat_completion(
            messages=messages,
            max_tokens=5000,
            model=model_path
        )
        print(response["choices"][0]["message"]["content"])
        return response

In [13]:
output = translate_paragraph(
    "褐蚁已经忘记这里曾是它的家园。这段时光对于暮色中的大地和刚刚出现的星星来说短得可以忽略不计，但对于它来说却是漫长的。",
    llm=llm,
    method="completion"
)

llama.cpp: loading model from /home/brian/github/llama.cpp/models/7B/Chinese-Alpaca-2/ggml-model-q4_0.bin
llama_model_load_internal: format     = ggjt v3 (latest)
llama_model_load_internal: n_vocab    = 55296
llama_model_load_internal: n_ctx      = 4096
llama_model_load_internal: n_embd     = 4096
llama_model_load_internal: n_mult     = 5504
llama_model_load_internal: n_head     = 32
llama_model_load_internal: n_head_kv  = 32
llama_model_load_internal: n_layer    = 32
llama_model_load_internal: n_rot      = 128
llama_model_load_internal: n_gqa      = 1
llama_model_load_internal: rnorm_eps  = 1.0e-06
llama_model_load_internal: n_ff       = 11008
llama_model_load_internal: freq_base  = 10000.0
llama_model_load_internal: freq_scale = 1
llama_model_load_internal: ftype      = 2 (mostly Q4_0)
llama_model_load_internal: model size = 7B
llama_model_load_internal: ggml ctx size =    0.08 MB
llama_model_load_internal: using CUDA for GPU acceleration
llama_model_load_internal: mem required  = 42

The brown ant has forgotten that this was once its home. To the earth and the newly rising stars, these moments are negligible; but to the ant, they lasted forever.



llama_print_timings:        load time =   594.70 ms
llama_print_timings:      sample time =    18.22 ms /    39 runs   (    0.47 ms per token,  2140.86 tokens per second)
llama_print_timings: prompt eval time =   594.68 ms /    55 tokens (   10.81 ms per token,    92.49 tokens per second)
llama_print_timings:        eval time =  3315.26 ms /    38 runs   (   87.24 ms per token,    11.46 tokens per second)
llama_print_timings:       total time =  3990.86 ms


In [5]:
output = translate_paragraph(
    "褐蚁继续沿着与地面平行的方向爬，进入了第三道沟槽，它是一个近似于直角的转弯，是这样的：“7”。它不喜欢这形状，平时，这种不平滑的、突然的转向，往往意味着危险和战斗。",
    llm=llm,
    use_example=True
)
print(output["choices"][0]["message"]["content"])

llama.cpp: loading model from /home/brian/github/llama.cpp/models/7B/Chinese-Alpaca-2/ggml-model-q4_0.bin
llama_model_load_internal: format     = ggjt v3 (latest)
llama_model_load_internal: n_vocab    = 55296
llama_model_load_internal: n_ctx      = 4096
llama_model_load_internal: n_embd     = 4096
llama_model_load_internal: n_mult     = 5504
llama_model_load_internal: n_head     = 32
llama_model_load_internal: n_head_kv  = 32
llama_model_load_internal: n_layer    = 32
llama_model_load_internal: n_rot      = 128
llama_model_load_internal: n_gqa      = 1
llama_model_load_internal: rnorm_eps  = 1.0e-06
llama_model_load_internal: n_ff       = 11008
llama_model_load_internal: freq_base  = 10000.0
llama_model_load_internal: freq_scale = 1
llama_model_load_internal: ftype      = 2 (mostly Q4_0)
llama_model_load_internal: model size = 7B
llama_model_load_internal: ggml ctx size =    0.08 MB
llama_model_load_internal: using CUDA for GPU acceleration
llama_model_load_internal: mem required  = 42

The brown ants continued to climb along a parallel direction with the ground, entering the third ditch. It was a nearly right-angled turn, as follows: “7”. The brown ants did not like this shape; usually, such abrupt and uneven turns mean danger and battle.



llama_print_timings:        load time =  1148.01 ms
llama_print_timings:      sample time =    29.15 ms /    63 runs   (    0.46 ms per token,  2161.53 tokens per second)
llama_print_timings: prompt eval time =  1147.96 ms /   345 tokens (    3.33 ms per token,   300.53 tokens per second)
llama_print_timings:        eval time =  5673.53 ms /    62 runs   (   91.51 ms per token,    10.93 tokens per second)
llama_print_timings:       total time =  6952.42 ms


In [ ]:
output

In [15]:
TEXT="在那个已被忘却的日子里，它的世界颠覆了。泥土飞走，出现了一条又深又宽的峡谷，然后泥土又轰隆隆地飞回来，峡谷消失了，在原来峡谷的尽头出现了一座黑色的孤峰。其实，在这片广阔的疆域上，这种事常常发生，泥土飞走又飞回，峡谷出现又消失，然后是孤峰降临，好像是给每次灾变打上一个醒目的标记。褐蚁和几百个同族带着幸存的蚁后向太阳落下的方向走了一段路，建立了新的帝国。"

In [16]:
output = translate_paragraph(
    TEXT,
    llm=llm,
    method="completion"
)


llama.cpp: loading model from /home/brian/github/llama.cpp/models/7B/Chinese-Alpaca-2/ggml-model-q4_0.bin
llama_model_load_internal: format     = ggjt v3 (latest)
llama_model_load_internal: n_vocab    = 55296
llama_model_load_internal: n_ctx      = 4096
llama_model_load_internal: n_embd     = 4096
llama_model_load_internal: n_mult     = 5504
llama_model_load_internal: n_head     = 32
llama_model_load_internal: n_head_kv  = 32
llama_model_load_internal: n_layer    = 32
llama_model_load_internal: n_rot      = 128
llama_model_load_internal: n_gqa      = 1
llama_model_load_internal: rnorm_eps  = 1.0e-06
llama_model_load_internal: n_ff       = 11008
llama_model_load_internal: freq_base  = 10000.0
llama_model_load_internal: freq_scale = 1
llama_model_load_internal: ftype      = 2 (mostly Q4_0)
llama_model_load_internal: model size = 7B
llama_model_load_internal: ggml ctx size =    0.08 MB
llama_model_load_internal: using CUDA for GPU acceleration
llama_model_load_internal: mem required  = 42

In that forgotten day, its world was transformed. Soil flew away, and a deep and wide valley appeared, then the soil returned and the valley disappeared at the end of it. It was a familiar sight to the vast wilderness: the soil flew away and returned, as well as the valley's appearance and disappearance. A black peak arrived there, like a flag in every disaster. Brown ants with hundreds of their tribe left behind walked towards the direction where the sun sets, and established a new empire.



llama_print_timings:        load time =   791.39 ms
llama_print_timings:      sample time =    49.42 ms /   111 runs   (    0.45 ms per token,  2246.15 tokens per second)
llama_print_timings: prompt eval time =   791.36 ms /   142 tokens (    5.57 ms per token,   179.44 tokens per second)
llama_print_timings:        eval time =  9651.34 ms /   110 runs   (   87.74 ms per token,    11.40 tokens per second)
llama_print_timings:       total time = 10666.27 ms


In [6]:

TEXT="破壁人二号（凄凉地笑笑）：“好了，都结束了，那边肯定是无梦的。”三体小说"

output = translate_paragraph(
    TEXT,
    llm=llm,
    use_example=True
)
print(output["choices"][0]["message"]["content"])
output

llama.cpp: loading model from /home/brian/github/llama.cpp/models/7B/Chinese-Alpaca-2/ggml-model-q4_0.bin
llama_model_load_internal: format     = ggjt v3 (latest)
llama_model_load_internal: n_vocab    = 55296
llama_model_load_internal: n_ctx      = 4096
llama_model_load_internal: n_embd     = 4096
llama_model_load_internal: n_mult     = 5504
llama_model_load_internal: n_head     = 32
llama_model_load_internal: n_head_kv  = 32
llama_model_load_internal: n_layer    = 32
llama_model_load_internal: n_rot      = 128
llama_model_load_internal: n_gqa      = 1
llama_model_load_internal: rnorm_eps  = 1.0e-06
llama_model_load_internal: n_ff       = 11008
llama_model_load_internal: freq_base  = 10000.0
llama_model_load_internal: freq_scale = 1
llama_model_load_internal: ftype      = 2 (mostly Q4_0)
llama_model_load_internal: model size = 7B
llama_model_load_internal: ggml ctx size =    0.08 MB
llama_model_load_internal: using CUDA for GPU acceleration
llama_model_load_internal: mem required  = 42

In that forgotten day, its world was turned upside down. The soil flew away and a deep and wide canyon appeared, then the soil flew back and the canyon disappeared, at the end of which appeared a black peak. In fact, such things often happened in this vast domain, where the soil flew away and flew back, then there was a canyon, then a peak, as if to mark each disaster. The brown ants and hundreds of their kind with surviving queens walked towards the direction where the sun set and established a new empire.



llama_print_timings:        load time =  1117.16 ms
llama_print_timings:      sample time =    58.16 ms /   127 runs   (    0.46 ms per token,  2183.44 tokens per second)
llama_print_timings: prompt eval time =  1117.11 ms /   319 tokens (    3.50 ms per token,   285.56 tokens per second)
llama_print_timings:        eval time = 11414.08 ms /   126 runs   (   90.59 ms per token,    11.04 tokens per second)
llama_print_timings:       total time = 12788.80 ms


{'id': 'chatcmpl-21a5571b-7b71-4089-af32-6839548c5ab5',
 'object': 'chat.completion',
 'created': 1692018905,
 'model': '/home/brian/github/llama.cpp/models/7B/Chinese-Alpaca-2/ggml-model-q4_0.bin',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': 'In that forgotten day, its world was turned upside down. The soil flew away and a deep and wide canyon appeared, then the soil flew back and the canyon disappeared, at the end of which appeared a black peak. In fact, such things often happened in this vast domain, where the soil flew away and flew back, then there was a canyon, then a peak, as if to mark each disaster. The brown ants and hundreds of their kind with surviving queens walked towards the direction where the sun set and established a new empire.'},
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 319,
  'completion_tokens': 127,
  'total_tokens': 446}}

In [18]:
tokenizer = LlamaTokenizer(llama=llm)

In [24]:
TEXT="在那个已被忘却的日子里，它的世界颠覆了。泥土飞走，出现了一条又深又宽的峡谷，然后泥土又轰隆隆地飞回来，峡谷消失了，在原来峡谷的尽头出现了一座黑色的孤峰。其实，在这片广阔的疆域上，这种事常常发生，泥土飞走又飞回，峡谷出现又消失，然后是孤峰降临，好像是给每次灾变打上一个醒目的标记。褐蚁和几百个同族带着幸存的蚁后向太阳落下的方向走了一段路，建立了新的帝国。"
tokens = tokenizer.encode(TEXT)

In [25]:
tokens

[1,
 30505,
 32380,
 36812,
 36183,
 32067,
 36162,
 30755,
 30214,
 33592,
 32109,
 40829,
 30743,
 30267,
 34230,
 31181,
 32255,
 32069,
 30214,
 34260,
 32915,
 32018,
 31947,
 32018,
 33162,
 30210,
 48207,
 30214,
 32088,
 34230,
 31181,
 32018,
 42303,
 31788,
 31788,
 30533,
 32255,
 33206,
 30214,
 48207,
 35030,
 30743,
 30214,
 30505,
 33419,
 48207,
 30210,
 47667,
 34260,
 35714,
 33760,
 30210,
 35841,
 32971,
 30267,
 32106,
 30214,
 35200,
 31122,
 42877,
 30210,
 39215,
 34872,
 30429,
 30214,
 43843,
 34826,
 32254,
 30214,
 34230,
 31181,
 32255,
 32069,
 32018,
 32255,
 30742,
 30214,
 48207,
 32143,
 32018,
 35030,
 30214,
 32088,
 30392,
 35841,
 32971,
 47617,
 30214,
 40941,
 31999,
 32827,
 35748,
 31462,
 31656,
 30429,
 32002,
 50121,
 30210,
 45962,
 30267,
 52694,
 49280,
 30503,
 37241,
 30502,
 30980,
 30999,
 33675,
 31762,
 30946,
 30210,
 49280,
 30822,
 31331,
 34109,
 32327,
 35096,
 32729,
 35002,
 33625,
 30874,
 30214,
 40783,
 33179,
 36231,
 302

In [27]:
for token in tokens:
    text = tokenizer.decode([token])
    print(text, end=" ")

 在 那个 已被 忘 却 的日子 里 ， 它的 世界 颠覆 了 。 泥 土 飞 走 ， 出现了 一条 又 深 又 宽 的 峡谷 ， 然后 泥 土 又 轰 隆 隆 地 飞 回来 ， 峡谷 消失 了 ， 在 原来 峡谷 的 尽头 出现了 一座 黑色 的 孤 峰 。 其实 ， 在这 片 广阔 的 疆 域 上 ， 这种事 常常 发生 ， 泥 土 飞 走 又 飞 回 ， 峡谷 出现 又 消失 ， 然后 是 孤 峰 降临 ， 好像是 给 每次 灾 变 打 上 一个 醒目 的 标记 。 褐 蚁 和 几百 个 同 族 带着 幸 存 的 蚁 后 向 太阳 落 下的 方向 走了 一段 路 ， 建立了 新的 帝国 。 

In [ ]:
TEXT="字幕：我们再进行第二项证实——伊文斯曾经给你发过一封加密信，但密码变了，他没来得及通知你新的密码就死了，你一直打不开那封信。现在我告诉你密码——CAMEL，就是你毒死金鱼的香烟的牌子。"